In [1]:
import requests
from bs4 import BeautifulSoup
import os
import time
try:
    from urllib.request import urlretrieve
except ImportError:
    from urllib import urlretrieve
import xml.sax
from sklearn import svm
import subprocess
import mwparserfromhell
import json
from collections import Counter
from itertools import chain
import numpy as np
import random
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Input, Reshape
from tensorflow.keras.layers import Dot
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import NearestNeighbors
import pickle
import gensim
from sklearn.decomposition import TruncatedSVD
import psycopg2

In [2]:
with open('data/wp_movies_10k.ndjson') as fin:
    movies = [json.loads(l) for l in fin]

In [3]:
link_counts = Counter()
for movie in movies:
    link_counts.update(movie[2])

top_links = [link for link, c in link_counts.items() if c >= 3]
link_to_idx = {link: idx for idx, link in enumerate(top_links)}
movie_to_idx = {movie[0]: idx for idx, movie in enumerate(movies)}
pairs = []
for movie in movies:
    pairs.extend((link_to_idx[link], movie_to_idx[movie[0]]) for link in movie[2] if link in link_to_idx)
pairs_set = set(pairs)
len(pairs), len(top_links), len(movie_to_idx)

(949544, 66913, 10000)

In [4]:
def movie_embedding_model(embedding_size=30):
    link = Input(name='link', shape=(1,))
    movie = Input(name='movie', shape=(1,))
    link_embedding = Embedding(name='link_embedding', input_dim=len(top_links), output_dim=embedding_size)(link)
    movie_embedding = Embedding(name='movie_embedding', input_dim=len(movie_to_idx), output_dim=embedding_size)(movie)
    dot = Dot(name='dot_product', normalize=True, axes=2)([link_embedding, movie_embedding])
    merged = Reshape((1,))(dot)
    model = Model(inputs=[link, movie], outputs=[merged])
    model.compile(optimizer='nadam', loss='mse')
    return model

model = movie_embedding_model()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [5]:
import random
random.seed(5)

def batchifier(pairs, positive_samples=50, negative_ratio=5):
    batch_size = positive_samples * (1 + negative_ratio)
    batch = np.zeros((batch_size, 3))
    while True:
        for idx, (link_id, movie_id) in enumerate(random.sample(pairs, positive_samples)):
            batch[idx, :] = (link_id, movie_id, 1)
        idx = positive_samples
        while idx < batch_size:
            movie_id = random.randrange(len(movie_to_idx))
            link_id = random.randrange(len(top_links))
            if not (link_id, movie_id) in pairs_set:
                batch[idx, :] = (link_id, movie_id, -1)
                idx += 1
        np.random.shuffle(batch)
        yield {'link': batch[:, 0], 'movie': batch[:, 1]}, batch[:, 2]

next(batchifier(pairs, positive_samples=3, negative_ratio=2))

({'link': array([ 1313., 32643., 22418., 31254., 48731., 20558., 13365.,  3801.,
         32318.]),
  'movie': array([7236., 7628., 1529., 5530., 1854.,  849., 6238., 5874., 7685.])},
 array([ 1., -1.,  1.,  1., -1., -1., -1., -1., -1.]))

In [6]:
positive_samples_per_batch=256

model.fit_generator(
    batchifier(pairs, positive_samples=positive_samples_per_batch, negative_ratio=5),
    epochs=10,
    steps_per_epoch=len(pairs) // positive_samples_per_batch,
    verbose=2
)

Epoch 1/10
Instructions for updating:
Use tf.cast instead.
 - 36s - loss: 0.9345
Epoch 2/10
 - 38s - loss: 0.4277
Epoch 3/10
 - 36s - loss: 0.3564
Epoch 4/10
 - 37s - loss: 0.3421
Epoch 5/10
 - 37s - loss: 0.3330
Epoch 6/10
 - 38s - loss: 0.3313
Epoch 7/10
 - 36s - loss: 0.3268
Epoch 8/10
 - 36s - loss: 0.3254
Epoch 9/10
 - 37s - loss: 0.3255
Epoch 10/10
 - 36s - loss: 0.3227


In [7]:
movie = model.get_layer('movie_embedding')
movie_weights = movie.get_weights()[0]
movie_lengths = np.linalg.norm(movie_weights, axis=1)
normalized_movies = (movie_weights.T / movie_lengths).T

def similar_movies(movie):
    dists = np.dot(normalized_movies, normalized_movies[movie_to_idx[movie]])
    closest = np.argsort(dists)[-10:]
    for c in reversed(closest):
        print(c, movies[c][0], dists[c])

similar_movies('Rogue One')

29 Rogue One 1.0
101 Prometheus (2012 film) 0.9631406
245 Gravity (film) 0.96278924
85 Inception 0.95210856
160 Jupiter Ascending 0.94889444
594 Dredd 0.94232994
25 Star Wars sequel trilogy 0.9389833
659 Rise of the Planet of the Apes 0.93705666
450 Dunkirk (2017 film) 0.93549836
19 Interstellar (film) 0.934783


In [8]:
movie = model.get_layer('movie_embedding')
movie_weights = movie.get_weights()[0]
movie_lengths = np.linalg.norm(movie_weights, axis=1)
normalized_movies = (movie_weights.T / movie_lengths).T
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(normalized_movies)

with open('data/movie_model.pkl', 'wb') as fout:
    pickle.dump({
        'nbrs': nbrs,
        'normalized_movies': normalized_movies,
        'movie_to_idx': movie_to_idx,
    }, fout)

In [9]:
with open('data/movie_model.pkl', 'rb') as fin:
    m = pickle.load(fin)
movie_names = [x[0] for x in sorted(movie_to_idx.items(), key=lambda t:t[1])]
distances, indices = m['nbrs'].kneighbors(
    [m['normalized_movies'][m['movie_to_idx']['Rogue One']]])
for idx in indices[0]:
    print(movie_names[idx])

Rogue One
Prometheus (2012 film)
Gravity (film)
Inception
Jupiter Ascending
Dredd
Star Wars sequel trilogy
Rise of the Planet of the Apes
Dunkirk (2017 film)
Interstellar (film)


In [13]:
DB_NAME = 'douwe'
USER = 'djangosite'
PWD = 'password'
#PWD = 'z0g3h31m!'
HOST = '127.0.0.1'
connection_str = "dbname='%s' user='%s' password='%s' host='%s'"
conn = psycopg2.connect(connection_str % (DB_NAME, USER, PWD, HOST))

In [11]:
with conn.cursor() as cursor:
    cursor.execute('INSERT INTO movie (movie_name, embedding) VALUES (%s, %s)',
                   (movie_names[0], normalized_movies[0].tolist()))
conn.commit()

IntegrityError: duplicate key value violates unique constraint "movie_pkey"
DETAIL:  Key (movie_name)=(Deadpool (film)) already exists.


In [14]:
with conn.cursor() as cursor:
    cursor.execute('DELETE FROM movie;')
conn.commit()

In [15]:
with conn.cursor() as cursor:
    for movie, embedding in zip(movies, normalized_movies):
        cursor.execute('INSERT INTO movie (movie_name, embedding)'
                       ' VALUES (%s, %s)',
               (movie[0], embedding.tolist()))
conn.commit()

In [16]:
conn.rollback()

In [17]:
def recommend_movies(conn, q):
    with conn.cursor() as cursor:
        cursor.execute('SELECT movie_name, embedding FROM movie'
                       '    WHERE lower(movie_name) LIKE %s'
                       '    LIMIT 1',
                       ('%' + q.lower() + '%',))
        if cursor.rowcount == 0:
            return []
        movie_name, embedding = cursor.fetchone()
        cursor.execute('SELECT movie_name, '
                       '       cube_distance(cube(embedding), '
                       '                     cube(%s)) as distance '
                       '    FROM movie'
                       '    ORDER BY distance'
                       '    LIMIT 5',
                       (embedding,))
        return list(cursor.fetchall())
    
recommend_movies(conn, 'The Force Awakens')

[('Star Wars: The Force Awakens', 1.33483561726229e-15),
 ('Jurassic World', 0.207172262120637),
 ('Interstellar (film)', 0.243386022049874),
 ('Guardians of the Galaxy (film)', 0.249480188212172),
 ('Doctor Strange (film)', 0.271664145410071)]

In [18]:
with conn.cursor() as cursor:
    cursor.execute('SELECT movie_name, cube_distance(cube(embedding), cube(%s)) as distance '
                   '    FROM movie'
                   '    ORDER BY distance'
                   '    LIMIT 5',
                   (emb,))
    x = list(cursor)
x

NameError: name 'emb' is not defined

In [ ]:
movies[0]

In [ ]:
MODEL = 'GoogleNews-vectors-negative300.bin'
model = gensim.models.KeyedVectors.load_word2vec_format(MODEL, binary=True)

In [ ]:
model.most_similar(positive=['espresso'])

In [ ]:
def most_similar(norm, positive):
    vec = norm[model.vocab[positive].index]
    dists = np.dot(norm, vec)
    most_extreme = np.argpartition(-dists, 10)[:10]
    res = ((model.index2word[idx], dists[idx]) for idx in most_extreme)
    return list(sorted(res, key=lambda t:t[1], reverse=True))

for word, score in most_similar(model.syn0norm, 'espresso'):
    print(word, score)

In [ ]:
svd = TruncatedSVD(n_components=100, random_state=42, n_iter=40)
reduced = svd.fit_transform(model.syn0norm)

In [ ]:
reduced_lengths = np.linalg.norm(reduced, axis=1)
normalized_reduced = (reduced.T / reduced_lengths).T
normalized_reduced.shape

In [ ]:
for word, score in most_similar(normalized_reduced, 'espresso'):
    print(word, score)

In [ ]:
for idx in most_extreme:
    print(model.index2word[idx], dists[idx])